# ch 19. naive bayes를 이용한 텍스트 분류

이번 챕터에서는 scikit learn에 내장된 NaiveBayesClassifier를 이용해서 실제 텍스트를 분류해보고, 성능을 측정해보겠습니다. 

## 데이터 셋 준비 

In [1]:
import pandas as pd

train_df = pd.read_csv("./data/naver_reviews_train.csv", encoding="utf-8")
test_df = pd.read_csv("./data/naver_reviews_test.csv", encoding="utf-8")


In [2]:
train_df

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 결측치 확인 및 제거

In [4]:
train_df.isnull().sum()
train_df = train_df.dropna()

In [5]:
test_df.isnull().sum()
test_df = test_df.dropna()

### 토큰화

In [6]:
from konlpy.tag import Komoran

komoran = Komoran()

In [7]:
komoran.morphs(train_df.iloc[0]["document"])

['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']

In [8]:
from tqdm import tqdm

tqdm.pandas()
train_df["tokens"] = train_df["document"].progress_apply(lambda x: komoran.morphs(x))
test_df["tokens"] = test_df["document"].progress_apply(lambda x: komoran.morphs(x))

100%|████████████████████████████████████████████████████████████████████████| 149995/149995 [01:44<00:00, 1439.68it/s]
C:\Users\82108\AppData\Local\Temp\ipykernel_18888\145535669.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["tokens"] = train_df["document"].progress_apply(lambda x: komoran.morphs(x))
100%|██████████████████████████████████████████████████████████████████████████| 49997/49997 [00:39<00:00, 1278.51it/s]


### CountVectorizer를 이용한 벡터화
Naive Bayes를 학습시키기 위해서는 문서 내 특정 단어의 빈도 수를 세어주어야 합니다. 이를 CountVectorizer를 이용해서 구현해보겠습니다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

vectorizer = CountVectorizer(
    analyzer = "word",
    tokenizer = dummy_fun,
    preprocessor = dummy_fun,
    token_pattern = None
)

In [10]:
vectorizer.fit(train_df["tokens"])

CountVectorizer(preprocessor=<function dummy_fun at 0x0000023B048BF560>,
                token_pattern=None,
                tokenizer=<function dummy_fun at 0x0000023B048BF560>)

In [11]:
x_train = vectorizer.transform(train_df["tokens"])

In [ ]:
## 저번에는 tfidf 사용했는데 이번에는 countvectorizer 빈도수만 계산

In [12]:
y_train = train_df["label"]

In [13]:
x_test = vectorizer.transform(test_df["tokens"])
y_test = test_df["label"]

## Naive Bayes Classifier 학습

scikit learn에 내장된 MultinomialNB를 이용하여 Naive Bayes Classifier를 학습시켜 보겠습니다. MultinomialNB를 사용하는 이유는 단어의 등장 빈도 수를 이용하여 분류하기 때문입니다. 먼저 학습시에 교차 검증으로 정확도를 측정하고, 전체 데이터 셋으로 다시 학습시킨 뒤, 테스트 셋에 대하여 정확도를 측정해보겠습니다.

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_val_score

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)


In [18]:
model = MultinomialNB()
scores = cross_val_score(model, x_train, y_train, cv=stratified_kfold)
print("교차 검증 정확도: ", scores)

교차 검증 정확도:  [0.82992766 0.82816094 0.8320944  0.83379446 0.82732758]


In [19]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [20]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("테스트 셋 정확도: ", accuracy)

테스트 셋 정확도:  0.82406944416665


In [21]:
## 인퍼런스??

## 정리

이번 챕터에서는 scikit learn에 내장된 MultinomialNB 클래스를 이용하여 텍스트 분류 모델을 학습시켜 보았습니다. 그 결과 82%로 상당히 준수한 성능이 나왔습니다. LSTM을 이용한 딥러닝 모델도 85% 정도 성능을 내는 것을 생각해보면 굉장히 가성비가 좋은 모델임을 알 수 있습니다. 베이즈 정리가 이렇게도 활용되는 구나 하는 점과 단순한 모델의 강력함을 꼭 기억하고 넘어갔으면 좋겠습니다.